# Basic analysis of data.

In today's class we will do some basic analysis of weather data.

Our source of data will be the <a href="http://climate.weather.gc.ca/historical_data/search_historic_data_e.html">Canadian Weather Database</a>.

Let's start by looking for the data from Victoria. 

Let's start by downloading the data and checking to see that we have a "full set". 

In [ ]:
## Let's start by loading all the weather data, storing it in a list of dictionaries. An element of 
## this list will consist of the weather file headers, turned into a dict-object.  

import datetime as dt
import os as os